In [1]:
import pandas as pd
import os
from google.cloud import bigquery

# Configuración de rutas
data_folder = "../../TFM/22_23_24/recuperaciones"
project_id = "banded-setting-428309-q4"
dataset_id = "datos"
table_id = "recuperaciones-data"

In [2]:
# Configuración de cliente de BigQuery
client = bigquery.Client(project=project_id)

In [3]:
# Función para cargar datos en BigQuery
def load_data_to_bigquery(df, table_id):
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("Timestamp", "TIMESTAMP"),
            bigquery.SchemaField("Value", "FLOAT"),
            bigquery.SchemaField("Tag", "STRING"),
        ],
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    )

    try:
        load_job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
        load_job.result()  # Esperar a que la carga termine
        print(f"Carga completada para la tabla {table_id}")
    except Exception as e:
        print(f"Error al cargar los datos en BigQuery: {e}")

In [4]:
# Verificar si la carpeta de datos existe
if os.path.exists(data_folder):
    print(f"Archivos en la carpeta de datos: {os.listdir(data_folder)}")
    for file in os.listdir(data_folder):
        if file.endswith(".csv"):
            file_path = os.path.join(data_folder, file)
            df = pd.read_csv(file_path)
            if all(column in df.columns for column in ["Timestamp", "Value", "Tag"]):
                print(f"Archivo {file} cargado.")
                try:
                    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d-%b-%y %H:%M:%S.%f")
                    load_data_to_bigquery(df, f"{project_id}.{dataset_id}.{table_id}")
                except Exception as e:
                    print(f"Error al procesar el archivo {file}: {e}")
            else:
                print(f"Archivo {file} no contiene las columnas requeridas y no se guardó.")
else:
    print(f"La carpeta de datos no existe: {data_folder}")

Archivos en la carpeta de datos: ['recup_1.csv', 'recup_2.csv']
Archivo recup_1.csv cargado.
Carga completada para la tabla banded-setting-428309-q4.datos.recuperaciones-data
Archivo recup_2.csv cargado.
Carga completada para la tabla banded-setting-428309-q4.datos.recuperaciones-data
